## Install Dependencies


In [ ]:
!pip install git+https://github.com/openai/whisper.git
!pip install pydub
!pip install git+https://github.com/snakers4/silero-vad.git

!pip install Groq
!pip install edge-tts
!pip install nest_asyncio


In [ ]:
!pip install webrtcvad

## Voice file to Text

In [ ]:
import torch

import torchaudio
import whisper


model = whisper.load_model("base")
wav_path = "/content/I%20am%20a%20big%20Brad%20Pitt.wav"
wav, sr = torchaudio.load(wav_path)
print (sr)

# Keeping Audio chanel count to 1
if wav.shape[0] > 1:
    wav = torch.mean(wav, dim=0, keepdim=True)

wav = torchaudio.transforms.Resample(orig_freq=sr, new_freq=16000)(wav)
# Applying noise reduction
denoiser = torchaudio.transforms.Vad(sample_rate=16000)
wav = denoiser(wav)




vad_model, utils = torch.hub.load(repo_or_dir='snakers4/silero-vad', model='silero_vad', force_reload=True)
get_speech_timestamps = utils[0]
# kkeping sr to 16000 and vad threshold to 0.5 as specified in problem statement
speech_timestamps = get_speech_timestamps(wav[0], vad_model, sampling_rate=16000, threshold=0.5)





transcription = ""
for i, segment in enumerate(speech_timestamps):
    start, end = segment['start'], segment['end']
    segment_wav = wav[:, start:end]


    temp_segment_path = f"temp_segment_{i}.wav"
    torchaudio.save(temp_segment_path, segment_wav, 16000)


    result = model.transcribe(temp_segment_path)
    transcription += result['text'] + " "


print(transcription)

## Text to LLM


In [ ]:
# Sending transcription to groq client using llama 3 llm for getting result
# also hardcoded the query to keep the output short and relevant


from groq import Groq


client = Groq(
    api_key="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",
)
transcript= transcription + "Please respond in short about 2-3 sentences without any bullet points  "
chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": transcript,
        }
    ],
    model="llama3-8b-8192",
)
answer= chat_completion.choices[0].message.content
print(answer)


## LLM output to Voice

In [7]:
import nest_asyncio
import edge_tts
import asyncio

nest_asyncio.apply()

async def text_to_speech(text, output_file, voice="en-US-JennyNeural", rate="+0%", pitch="0Hz"):

    communicate = edge_tts.Communicate(text, voice, rate=rate, pitch=pitch)
    await communicate.save(output_file)

# Define your text
text = ( answer)
# Choose your voice (e.g., en-US-JennyNeural for female, en-US-GuyNeural for male)
voice = "en-US-GuyNeural"

# Adjust the speed (rate) and pitch
rate = "+10%"
pitch = "+1Hz"


output_file = "output_test_test_2.mp3"


asyncio.run(text_to_speech(text, output_file, voice=voice, rate=rate, pitch=pitch))
